The explanations and what I will apply is from and after reading ***A neural probabilistic language model paper*** *(Bengio, Y., et al, 2003)*

So we learned about n-grams language models, and how we can predict the next word given the $n$ previous words. And we implemented a **very simple** bigram model.  
Now, my next step was to understand word embeddings is how n-grams and statistical language modeling translated into neural networks and deep learning.

So in *(Bengio, Y., et al, 2003)*, the motivations they had is that it's difficult to model the joint probablity distribution of multiple consecutive words (random variables with **discrete** values.) An example would be having a sentence of 10 words where each one could have any value of a vocabulary of $100000$ words, so you have about $100000^{10}$ different possibilities or as they say "free parameters" for that sentence.

They also explained that modeling continuous variables is easier for generalization because the approximation function we want to learn is expected to **have** some smooth properties (neural networks would be a good example.) So the continuous space has a smoother space unlike discrete spaces where are more, idk, **discrete?** And they explain more about discrete spaces:  
***"For discrete spaces, the generalization structure is not as obvious: any change of these discrete variables may have a drastic impact on the value of the function to be estimated, and when the number of values that each discrete variable can take is large, most observed objects are almost maximally far from each other in hamming distance."***

**<u>Disclaimer:</u>**  
We are now going into the approach proposed and the mathematics and the design of it. Since I myself am trying to learn here, I won't be following the paper exactly here, meaning that I could:  
1.  Add a little bit more mathematics to give you and myself a better idea of what's happening.  
2.  Branch quickly in what I am writing to explain some other concepts that will help us understand what we have here better, which might be trivial to you depending on your level and knowledge so you can skip. But it won't be long anyway, this isn't an article (yet!)  

So previously, we learned that you can predict the next word $w_t$ given all previous ones:  
    $$P(w_t | w_{1:t-1}) \tag{1}$$  
    
But in n-grams, we wanted to approximate $(1)$ by using just the $n$ words before $w_t$ as follows:  
    $$P(w_t | w_{t-n+1:t-1}) \approx P(w_t | w_{1:t-1}) \tag{2}$$

Remember the joint probability equation, assuming there's a dependance between variables $x$, $y$, and $z$ (which in our case are words):
    $$P(x \cap y \cap z) = P(x, y, z) = P(z) P(y | z) P(x | z, y) \tag{3}$$
Which can be generalized to:
    $$P(x_i, x_{i-1}...x_{1}) = \prod_{k = 1}^n{P(x_k | x_1, x_2...x_{k-1})} \tag{4}$$

And now that we can approximate the probability of $w_t$, we can get the joint probability of a sentence or a sequence of words $W$ of length $T$:  

$$P(w_t w_{t-1}...w_1) = \prod_{k=1}^T{P(w_k | w_{k-n+1:k-1})} \tag{5}$$

This is the function we would want to maximize.

Side note for the joint probability in $(3)$: if we assumed there's no dependance between $x$, $y$, and $z$, we have:
    $$P(x, y, z) = P(x) P(y) P(z) \tag{6}$$

We are now want to use neural network to find a good function $f$ that approximates this joint probability distribution for a specific corpus. $f$ would have a set of trainable parameters $\Theta$.  
Now from above and $(2)$:  
$$P(w_t | w_{t-n+1:t-1}) \approx P(w_t | w_{1:t-1}) \newline
\approx f(w_t, w_{t-1}...w_{t-n+1};\Theta) \tag{7}$$

You can notice that $f$ uses $w_t$ and the n previous words $(w_{t-1}...w{t-n+1})$, just like we do in n-grams modelling.  
Now, from $(5)$ & $(7)$, we want to get the joint probability of the whole word sequence $W$ of word count $T$:
$$P(w_t w_{t-1}...w_1) = \prod_{k=1}^T{P(w_k | w_{k-n+1:k-1})} \newline
\approx \prod_{t = 1}^T{f(w_t, w_{t-1}...w_{t-n+1};\Theta)} \tag{8}$$  

<u>What are the parameters included in $\Theta$?</u>

The parameters we have would depend on the model's architecture, we have mainly 2 parts (according to the architecture proposed in the text.):
1. Assuming we would have a set of vocabulary $V$ of size $|V|$, we want to create a mapping (a lookup table) $C$ for each word $w_i$ to a vector $v_{w_i}$ whose dimensions $\in \mathbb{R}^m$. In other words: $C(w_i) = v_{w_i} \mid v_{w_i} \in \mathbb{R}^m$.
2. The probability function $g$ that takes the mapping $C$ of the input words and maps them to the probability distribution over the vocab $V$.
    $$f(w_t, w_{t-1}...w_{t-n+1};\Theta) = g(i, C(w_{t-1}...w_{t-n+1}))$$

So $g$ outputs a vector of $|V|$ dimensions ($\in \mathbb{R}^{|V|}$), where the $i_{th}$ dimension or element corresponds to the probability of $i_{th}$ word being the next token:
$$\hat{P}(w_t=i | w_{1:t-1})$$

Then $C$ would be an $|V| \times m$ matrix. You can also say that $C$ is the word embeddings as a term we know of today.

We now want to start definining the loss function, which will be equivalent to $(8)$:
$$l = \prod_{t = 1}^T{f(w_t, w_{t-1}...w_{t-n+1};\Theta)} \tag{9}$$

We can transform the product into a summation by taking the log of both sides:
$$log(l) = log(\prod_{t = 1}^T{f(w_t, w_{t-1}...w_{t-n+1};\Theta)}) \newline
= \sum_{t = 1}^T{log(f(w_t, w_{t-1}...w_{t-n+1};\Theta))} \tag{10}$$
**<u>Why do we take the log:</u>**  
* Multiplying a lot of probabilities would make the resulting value approach $0$, or a numerical underflow when running on a computer which can mess up the calculations and results.
* Likewise if we were multiplying large numbers would result in an even bigger result, eventually leading to an overflow.

What we reached in $(10)$ is what we call Log Likelihood. So, from $(10)$, our loss function would be:
$$L = \sum_{t = 1}^T{log(f(w_t, w_{t-1}...w_{t-n+1};\Theta))} \tag{11}$$

We can also add a Regularization function $R$ that regularizes the set of parameters $\Theta$, or a subset of $\Theta$ called $\Omega$:
$$L = \sum_{t = 1}^T{log(f(w_t, w_{t-1}...w_{t-n+1};\Theta))} + R(\Omega) \tag{12}$$

But I won't use the regularized term $R(\Omega)$ in this implementation, so we will use $(11)$ as our Loss Function for the implementation.

Now for the network's architecture:  
<p align="center">
    <img src="../images/model_architecture.jpeg" alt="Bengio, et al. 2003" width="600"/>
</p>  

The first layer is the embeddings layer, then we have just one hidden layer before the $Softmax$ function. They also describe that **we can optionally connect the embeddings layer directly to the softmax layer.**

**<u>What what are the parameters we have?</u>**  

$$\Theta = (b, d, W, U, H, C)$$

| Parameter | Size | Optional | Description |
|:----------:|:----------:|:----------:|:----------:|
| $C$    | $\|V\| \times m$   | No   | The lookup table for the vector representation of each word in the vocabulary $\|V\|$.   |
| $H$    | $h \times (n-1) m$  | No   | The weights matrix for the hidden layer.   |
| $U$    | $\|V\| \times h$  | No   | The weights matrix for the hidden layer output to the softmax layer.   |
| $b$    | $\|V\|$  | No   | The output layer's biases.   |
| $d$    | $h$  | No   | The hidden layer's biases.   |
| $W$    | $\|V\| \times (n-1)m$  | Yes   | The weights matrix for optional connection between the embeddings layer and softmax layer.   |

Such that the logits of the output layer y that's passed to the softmax activation:
$$y = b + Wx + U tanh(d + Hx)$$

The softmax activation (predicted probability of word $w_t$ given the previous $n$):
$$\hat{P}(w_t | w_{t-1:t-n+1}) = \frac{\textit{e}^{y_{w_t}}}{\sum_{i}{\textit{e}^{y_i}}}$$


Perhaps that's enough reading, lets try to make a naive implementation of the architecture above with ***PyTorch***.

In [ ]:
import torch
from torch import nn

# Our Neural Language Model's class
class NLM(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        

In [16]:
import nltk
from nltk.corpus import brown
nltk.download('brown')
brown

[nltk_data] Downloading package brown to /Users/shadyali/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


<CategorizedTaggedCorpusReader in '/Users/shadyali/nltk_data/corpora/brown'>

In [20]:
brown.sents()

[['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.'], ['The', 'jury', 'further', 'said', 'in', 'term-end', 'presentments', 'that', 'the', 'City', 'Executive', 'Committee', ',', 'which', 'had', 'over-all', 'charge', 'of', 'the', 'election', ',', '``', 'deserves', 'the', 'praise', 'and', 'thanks', 'of', 'the', 'City', 'of', 'Atlanta', "''", 'for', 'the', 'manner', 'in', 'which', 'the', 'election', 'was', 'conducted', '.'], ...]

In [25]:
len(brown.words())

1161192

In [26]:
len(set(brown.words()))

56057